In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import plotly.express as px
import os
from datetime import datetime

## Set ranges for OFAT analysis and defaults values for all parameters.

In [4]:
dict_of_ofat = {'const_dividend_earnings_ratio': [i/10 for i in range(0, 11, 1)],
               'const_income_tax_rate': [i/100 for i in range(0, 91, 2)],
               'ecb_interest_rate' : [i/100 for i in range(-6, 25, 2)],
               'gov_policy_swith_quantitative_easing' : [0,1]}

default_values = {'const_dividend_earnings_ratio': 0.7,
                 'const_income_tax_rate' : 0.05,
                 'ecb_interest_rate' : 0.05,
                 'gov_policy_swith_quantitative_easing' : 0}

This block of code copies initial file to a folder for OFAT experiment.\
Then it changes parameters' values in the file to those we need for OFAT.

**Note:** values you want to change for OFAT analysis should be set to 0 in the initial file. I use sed for setting of proper values for OFAT. This is fine because I worked with environmental variables that are listed in the file in a single copy.

**TODO:** Use XML parser for setting values without sed.\
**TODO2:** Add a parallel execution of this code. I've been using a couple of Jupyter instances instead, because the bottleneck was the disk capacity but not the execution speed.\
**TODO3:** Make the model save only those variables, that you need. This will reduce the execution time. Now writing all this info to a disk takes a lot of time. Also a capacity is a problem. 

In [ ]:
path = '/data/home/nsov/eurace/original/reproduce_results/'
path_to_initial = '/data/home/nsov/eurace/original/trial2/its/cbs_new_stable_point.xml'


number_of_years = 2
days = 20 * 12 * number_of_years

global_number = 0

print(datetime.now().strftime("%H:%M:%S"))

for param, values in dict_of_ofat.items():
    for case in values:
        local_ofat_dict = default_values.copy()
        local_ofat_dict[param] = case
        for i in range(10,15):
            global_number += 1
            current_dir = path+"test_div_{number}".format(number= global_number)
            try:
                os.mkdir(current_dir)
            except OSError:
                print('ERROR: Failed to create {cur_dir}'.format(cur_dir = current_dir))
            os.system('cp {init_file} {target}/0_initial.xml'.format(init_file = path_to_initial, target = current_dir))
            os.system('sed -i -e "s/<GSL_RNG_SEED>0<\/GSL_RNG_SEED>/<GSL_RNG_SEED>{num}<\/GSL_RNG_SEED>/" {cur_dir}/0_initial.xml'.format(num = int(i), cur_dir = current_dir))
            for parameter, set_value in local_ofat_dict.items():
                os.system('sed -i -e "s/<{parameter}>0<\/{parameter}>/<{parameter}>{set_value}<\/{parameter}>/" {cur_dir}/0_initial.xml'.format(parameter = parameter, set_value = set_value, cur_dir = current_dir))
            os.system('/data/home/nsov/eurace/original/trial2/main {days} {cur_dir}/0_initial.xml -f 20'.format(days = days, cur_dir=current_dir))
        print('Done for '+current_dir)
        print(datetime.now().strftime("%H:%M:%S"))
